# Error-Type-Aware Analysis for SupervisedER Task

This notebook runs per-error-type performance analysis for the SupervisedER task.

## Setup


In [ ]:
# Clone repository (if not already cloned)
# !git clone <repository_url>
# %cd <repository_directory>

# Or if already in the repository directory, just change to it
import os
os.chdir('/content/AML')  # Adjust path as needed


In [ ]:
# Install requirements
%pip install -q numpy pandas scikit-learn tqdm loguru matplotlib seaborn torcheval pyrebase4 yacs

# Install PyTorch (adjust for your Colab environment)
# For CUDA:
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# For CPU:
# %pip install torch torchvision torchaudio


In [ ]:
# Mount Google Drive (optional - if checkpoints/data are stored there)
from google.colab import drive
drive.mount('/content/drive')

# Example: Copy checkpoints from Drive
# !cp -r /content/drive/MyDrive/AML/checkpoints ./checkpoints


## Run Error-Type Analysis

Run the analysis script with your model checkpoint and configuration.


In [ ]:
# Example: Run analysis for MLP variant with Omnivore backbone on step split
# Adjust parameters based on your checkpoint

!python analysis/error_type_analysis.py \
    --split step \
    --backbone omnivore \
    --variant MLP \
    --ckpt checkpoints/error_recognition_best/MLP/omnivore/error_recognition_MLP_omnivore_step_epoch_43.pt \
    --threshold 0.6 \
    --output-dir analysis/outputs


In [ ]:
# Example: Run analysis for Transformer variant with Omnivore backbone on step split

!python analysis/error_type_analysis.py \
    --split step \
    --backbone omnivore \
    --variant Transformer \
    --ckpt checkpoints/error_recognition_best/Transformer/omnivore/error_recognition_Transformer_omnivore_step_epoch_XX.pt \
    --threshold 0.6 \
    --output-dir analysis/outputs


In [ ]:
# Example: Run analysis for recordings split (use threshold 0.4)

!python analysis/error_type_analysis.py \
    --split recordings \
    --backbone omnivore \
    --variant MLP \
    --ckpt checkpoints/error_recognition_best/MLP/omnivore/error_recognition_MLP_omnivore_recordings_epoch_XX.pt \
    --threshold 0.4 \
    --output-dir analysis/outputs


## Display Results

Load and display the results.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load results CSV
results_df = pd.read_csv('analysis/outputs/error_type_analysis_step_omnivore_MLP_threshold_0.6.csv')

# Display table
print("Per-Error-Type Performance Metrics:")
print("=" * 80)
display(results_df)


In [ ]:
# Display saved plots
from IPython.display import Image, display

fig_path = 'analysis/outputs/error_type_analysis_step_omnivore_MLP_threshold_0.6.png'
if os.path.exists(fig_path):
    display(Image(fig_path))
else:
    print(f"Plot not found at {fig_path}")
